In [1]:
#!pip install fasttext
!pip install cltk
import pandas as pd
import numpy as np
import json
import os
#from stemmer import *
#import fasttext
import spacy
nlp = spacy.load('en_core_web_sm')
import re
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
!pip install laserembeddings
!python -m laserembeddings download-models

  ERROR: Could not find a version that satisfies the requirement torch<2.0.0,>=1.0.1.post2 (from laserembeddings) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch<2.0.0,>=1.0.1.post2 (from laserembeddings)
C:\Users\midas\AppData\Local\Continuum\anaconda3\python.exe: No module named laserembeddings


In [2]:
from cltk.corpus.utils.importer import CorpusImporter
from cltk.tokenize.sentence import TokenizeSentence
from cltk.tokenize.word import WordTokenizer
c = CorpusImporter('hindi')
tokenizer = TokenizeSentence('hindi')
word_tokenizer = WordTokenizer('multilingual')


In [3]:
df=pd.read_csv("Total_data_annotated.csv")
#del(df["Unnamed: 0"])

In [4]:
df.head()

,Unnamed: 0,Mithun,Punyajoy,Soumyadeep,id,label,text,translated
0,0,0,1,0,524726,0.0,आज ही के दिन 30 जनबरी को नाथूराम ने गाँधी वध क...,NaN
1,1,0,0,0,207102,0.0,"*👉जम्मू कश्मीरः बडगाम में एनकाउंटर🔫 जारी, दो आ...",* 👉 Jammu Kashmir: Encounter continues 🔫 in ...
2,2,0,0,0,135627,0.0,*यह साध्वी जी की परिपक्वता और गहरी समझ की निशा...,* It is a sign of maturity and deep understand...
3,3,0,0,0,665173,0.0,"भारतीय सेना ने की एक और सर्जिकल स्‍ट्राइक, म्‍...",The Indian Army carried out this operation in ...
4,4,0,1,0,543553,0.0,एक नया ड्रामा डॉ अम्बेडकर के नाम पर प्रचलित हु...,A new drama has become popular in the name of ...


In [7]:
with open('Resource/stopwords-hi.json','r',encoding="utf8") as fp:
  HindiStopwords=json.load(fp)

In [8]:
EnglishStopwords=nlp.Defaults.stop_words

In [9]:
mixStopWords = HindiStopwords
mixStopWords.extend(EnglishStopwords)

In [25]:
#PRETRAINED_MODEL_PATH = 'Resource/lid.176.bin'
#model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [10]:
def tokenizeText(text):
  #predictions = model.predict(text)
  removedLinkText=re.sub(r"http\S+", "", text)
  doc=word_tokenizer.tokenize(removedLinkText)
  return doc

def preprocessed(TokeninzedSent):
  Processedlist=[]
  for tokens in TokeninzedSent:
    temp=[]
    for token in tokens:
      if(len(token)>2):
        temp.append(token)
    Processedlist.append(temp)
  return Processedlist

def RemoveStopwords(WordList,Stopwords):
  Processedlist=[]
  for tokens in WordList:
    temp=[]
    for token in tokens:
      if(token not in Stopwords):
        temp.append(token)
    Processedlist.append(temp)
  return Processedlist 


In [11]:
TokeninzedSent=[]
label=[]
for i in range(0, len(df)):
  TokeninzedSent.append(tokenizeText(df['text'][i]))
  label.append(df['label'][i])
print(TokeninzedSent[1])

['*👉जम्मू', 'कश्मीरः', 'बडगाम', 'में', 'एनकाउंटर🔫', 'जारी', ',', 'दो', 'आतंकी👥', 'ढेर', ',', 'चार', 'जवान', 'घायल🤕*', 'जम्मू', 'कश्मीर', ',', 'बडगाम', 'जिले', 'के', 'सुत्सू', 'गांव', 'में', 'सुरक्षाबलों', 'और', 'आतंकियों', 'के', 'बीच', 'मुठभेड़', 'में', 'दो', 'आतंकियों', 'के', 'मारे', 'जाने', 'की', 'खबर', 'सामने', 'आई', 'है।', 'अधिकारियों', 'ने', 'इस', 'बारे', 'में', 'जानकारी', 'दी', 'है।', 'एएनआई', 'की', 'रिपोर्ट', 'के', 'मुताबिक', 'एनकाउंटर', 'में', 'दो', 'आतंकी', 'मारे', 'गए', 'हैं।', 'मुठभेड़', 'के', 'दौरान', 'चार', 'जवान', 'भी', 'घायल', 'हुए', 'हैं।', 'यह', 'एनकाउंटर', 'अभी', 'भी', 'जारी', 'है।', 'इलाके', 'के', 'एक', 'घर', 'में', 'कम', 'से', 'कम', 'तीन', 'आतंकियों', 'के', 'छिपे', 'होने', 'की', 'आशंका', 'है।', 'इस', 'खबर', 'के', 'बारे', 'में', 'ज्यादा', 'जानकारी', 'का', 'इंतजार', 'है।', 'बीते', 'गुरुवार', 'को', 'भी', 'जम्मू-कश्मीर', 'के', 'शोपियां', 'और', 'कुपवाड़ा', 'जिलों', 'में', 'सुरक्षा', 'बलों', 'के', 'साथ', 'दो', 'मुठभेड़ों', 'में', 'चार', 'आतंकवादी', 'मारे', 'गए', 'थे।', 'प

In [12]:
print(len(TokeninzedSent))
print(len(label))

540
540


In [13]:
processedList=preprocessed(TokeninzedSent)
FinalprocessedList=RemoveStopwords(processedList,mixStopWords)

In [14]:
FinalList=[]
for lst in FinalprocessedList:
  FinalList.append(' '.join(lst))

In [15]:
FinalList[0]

'दिन जनबरी नाथूराम गाँधी अन्यथा गाँधी योजना पाकिस्तान जाने महात्मा बनने गाँधी पाकिस्तान मांगे मानने वाला देश टुकड़े तैयार ... नाथूराम जिनकी गाँधी व्यक्तिगत दुश्मनी महात्मा नाथूराम अच्छे परिवार अमीर महाराष्ट्र चलकर दिल्ली पहुंचे दिन यानि जनवरी महात्मा नाथूराम गाँधी देश बचा लिया गर्व देश बचा नाथूराम देश हिस्सों तोड़ने पूर्वी पाकिस्तान पश्चिमी पाकिस्तान रास्ता बनाने मंशा कारण गाँधी बता दें पूर्वी पाकिस्तान पश्चिमी पाकिस्तान बनाया गाँधी सेकुलरिज्म चक्कर जिहादियों बात माना जिसमे जिहादी चाह पूर्वी पाकिस्तान लेकर पश्चिमी पाकिस्तान जाने भारत बीचों बीच किलोमीटर चौड़ा रास्ता जाये रस्ते केवल मुसलमानो रहने इज़ाज़त जाए ताकि रास्ता पूर्वी पाकिस्तान पश्चिमी पाकिस्तान आने जाने मुसलमानो सुरक्षित गाँधी मुसलमानो मांग मानने वार्ता पश्चिमी पाकिस्तान जाने वाला महात्मा नाथूराम गांधीवध देश बचा लिया महात्मा नाथूराम गोडसे ऐसा नक्शा भारत ... पीली लाइन देख किलोमीटर चौड़ा रास्ता पूर्वी पाकिस्तान पश्चिमी पाकिस्तान जाना किलोमीटर केवल मुस्लिम रास्ता पंजाब/राजस्थान यूपी बिहार बंगाल अन्य पूर्वी पाकिस्तान जाना नाथूराम प्राणों

In [16]:
newDataFrame=pd.DataFrame()
newDataFrame['text']=FinalList
newDataFrame['label']=label

In [33]:
newDataFrame.head()

,text,label
0,दिन जनबरी नाथूराम गाँधी अन्यथा गाँधी योजना पाक...,0.0
1,*👉जम्मू कश्मीरः बडगाम एनकाउंटर🔫 जारी आतंकी👥 ढे...,0.0
2,*यह साध्वी परिपक्वता गहरी समझ निशानी है। उन्हो...,0.0
3,भारतीय सेना सर्जिकल स्‍ट्राइक म्‍यांमार सेना म...,0.0
4,नया ड्रामा अम्बेडकर नाम प्रचलित उद्देश्य केवल ...,0.0


     |████████████████████████████████| 860kB 18.6MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=982d4fd860256c44255a556fa0a58a8bfc00170629e39ed87fbe74274cd59c94
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [0]:
from laserembeddings import Laser
laser = Laser()

In [0]:
X0 = laser.embed_sentences(newDataFrame['text'],lang='hi')
y0 = newDataFrame['label'].values

In [39]:
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
class_weights = dict(zip(np.unique(y0), compute_class_weight("balanced", np.unique(y0),y0)))
print(class_weights)
classifier= LogisticRegression(class_weight=class_weights)
#classifier= LogisticRegression()

{0.0: 0.7336956521739131, 1.0: 1.569767441860465}


In [0]:
accuracy = cross_val_score(classifier, X0, y0, cv=10)
scores = cross_val_score(classifier, X0, y0, scoring='f1_macro', cv=10)


In [41]:
print("Accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std() * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.68 (+/- 0.08)
Macro F1: 0.66 (+/- 0.09)


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [19]:
X1 = cv.fit_transform(newDataFrame['text']).toarray()
y1 = newDataFrame['label'].values

In [44]:
!pip install sklearn

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
class_weights = dict(zip(np.unique(y1), compute_class_weight("balanced", np.unique(y1),y1)))
print(class_weights)
classifier= LogisticRegression(class_weight=class_weights)
#classifier= LogisticRegression()

{0.0: 0.7336956521739131, 1.0: 1.569767441860465}


In [24]:
accuracy = cross_val_score(classifier, X1, y1, cv=10)
scores = cross_val_score(classifier, X1, y1, scoring='f1_macro', cv=10)

C:\Users\midas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\midas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\midas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\midas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\midas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linea

In [25]:
print("Accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std() * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.76 (+/- 0.15)
Macro F1: 0.73 (+/- 0.16)
